In [1]:
import matplotlib.pyplot as plt
import polars as pl
import numpy as np

# change the configsetting, to see the full tables
pl.Config.set_tbl_rows(7)
pl.Config(fmt_str_lengths=550)

df = pl.read_parquet("../data/MINES/reactions_compounds_list.parquet")

In [2]:
# Filter out rows with empty lists
df_filtered = df.filter((pl.col("starting_compounds").list.len() > 0) & (pl.col("predicted_compounds").list.len() > 0))

# Ensure both columns have matching element counts
df_exploded = df_filtered.explode(['starting_compounds']).explode(['predicted_compounds'])

df_exploded

reaction_id,starting_compounds,predicted_compounds
str,str,str
"""R5d8539f1d9a5e857189956bad8eb48e7b11137a25b12c2a7293fa1b61562e629""","""Ce8bc5cd3aa30776ab6d35fdc2bcc4707f4ac2919""","""C878f017efe6de2805a953d0ca9b8491274a29290"""
"""R6a89aaf90529aa474f537c081d71ff584fa0179bdde8269c9ae4b80b2e5c96f7""","""Ce8bc5cd3aa30776ab6d35fdc2bcc4707f4ac2919""","""C54130f1c76aaa5380fa631a6a659121284978c5d"""
"""Rf0c39549766c89963dbb1a98f8f1d4b89431cf4ee60f962c7703ef9df5bcb327""","""Ce8bc5cd3aa30776ab6d35fdc2bcc4707f4ac2919""","""Cfa5e885b86c8c37a465cad5238ed62672498a45d"""
"""Rad931d485dae8cbc9aa07c1301163f00258690adddbc7279de2b5c92ebca656f""","""Ce8bc5cd3aa30776ab6d35fdc2bcc4707f4ac2919""","""C57a73b796ef9de341670ad4f895779c4ce0d4623"""
…,…,…
"""Rc52c2d326695e43c104ecd4a11b20ead83987b49c39bbc0c165c7d25b8fef8b5""","""C92af25813ba38f4e1f385dc92b0e253dc7a01f6c""","""C5e8022bf3b0f1ee82b5ec45f5c89f69ea2ab5a29"""
"""Rb993c3707a42c88a143334e22d4af7eacf0dde11657106b272262f033e17d424""","""C326b3a90bc4bff0d2cf249e40ce517af77a62d0d""","""Ca4ef9345ff26e4fec21cc7fb72fa9dca2f86fcc5"""
"""Rc1fdf76ef8f34b7c3a7538fcb9924dd3a44615fd990b2da338cc61b791734e6d""","""C62f5460835a65fcb75c6c8da0f7f6f90c060d900""","""Ce9a6758cc6c58ca618292cde869df568a1c34cb8"""


In [3]:
print( df_exploded.describe() )

print("reaction_id:", df_exploded[:, 'reaction_id'].unique().count())
print("starting compounds: ", df_exploded[:, 'starting_compounds'].unique().count())
print("predicted_compounds:", df_exploded[:, 'predicted_compounds'].unique().count())


shape: (9, 4)
┌────────────┬────────────────────────────┬────────────────────────────┬───────────────────────────┐
│ statistic  ┆ reaction_id                ┆ starting_compounds         ┆ predicted_compounds       │
│ ---        ┆ ---                        ┆ ---                        ┆ ---                       │
│ str        ┆ str                        ┆ str                        ┆ str                       │
╞════════════╪════════════════════════════╪════════════════════════════╪═══════════════════════════╡
│ count      ┆ 3455876                    ┆ 3455876                    ┆ 3455876                   │
│ null_count ┆ 0                          ┆ 0                          ┆ 0                         │
│ mean       ┆ null                       ┆ null                       ┆ null                      │
│ std        ┆ null                       ┆ null                       ┆ null                      │
│ …          ┆ …                          ┆ …                          ┆ …   

In [4]:
# Create a directed graph
G = nx.DiGraph()

# Convert the DataFrame to a list of dictionaries
rows = df_exploded.to_dicts()

# Add edges to the graph
for row in rows:
    starting_compound = row['starting_compounds']
    predicted_compound = row['predicted_compounds']
    G.add_edge(starting_compound, predicted_compound)

# Draw the graph
plt.figure(figsize=(12, 8))
pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True, node_size=3000, node_color="skyblue", font_size=10, font_color="black", font_weight="bold", edge_color="gray")
plt.title('Compound Prediction Network')
plt.show()

NameError: name 'nx' is not defined